In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data=pd.read_csv('../input/twitter-sentiment/train_2kmZucJ.csv')
data_test=pd.read_csv('../input/test-data/test_oJQbWVk.csv')

In [ ]:
data.info()


In [ ]:
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [ ]:
def preprocesstweet(tweet):
    st1=re.sub('http://\S+' or ' http://\S+'," ",tweet)
    st2=re.sub('[#,@]\S+'," ",st1 ).lower()
    st3=re.sub('[^a-zA-Z]'," ",st2).split()
    st4=[]
    for word in st3:
        if word in (stopwords.words('english')):
            pass
        else:
            st4.append(word)
    st5=[]
    for stm in st4:
        st5.append(ps.stem(stm))
        tweet = ' '.join(st5)
    return tweet        

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
BoW = CountVectorizer()
TFIDF_table = TfidfVectorizer()


In [ ]:
BoW_table=BoW.fit_transform(data['tweet'].apply(preprocesstweet))
BoW_table

In [ ]:
BoW_table_test=BoW.transform(data_test['tweet'].apply(preprocesstweet))
BoW_table_test

In [ ]:
x_sub=BoW_table_test.toarray()
x_sub.shape

In [ ]:
x=BoW_table.toarray()
x.shape


In [ ]:
y = data['label']
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=56)
model.fit(x_train,y_train)
model.score(x_train,y_train)
model.score(x_test,y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


In [ ]:
sv=SVC(kernel='linear')
dec=DecisionTreeClassifier(max_depth=10)
rad=RandomForestClassifier(max_depth=10)
xg=XGBClassifier(max_depth=5)
kn=KNeighborsClassifier()
nb=GaussianNB()
mod=[sv,dec,rad,kn,nb]

In [ ]:
accuracy=[]
def ml(mod):
    for model in mod:
        model.fit(x_train,y_train)
        model.score(x_train,y_train)
        acc=model.score(x_test,y_test)
        accuracy.append(acc)
        y_pre=model.predict(x_test)

In [ ]:
ml(mod)

In [ ]:
accuracy

In [ ]:
x_train.reshape(1,5940,8763)
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM

In [ ]:

model_ann=Sequential()
#model_ann.add(Dense(1,activation='relu',input_shape=(8763,)))
model_ann.add(LSTM(32))

#model_ann.add(Dense(110,activation='relu'))
#model_ann.add(Dense(10,activation='relu'))
model_ann.add(Dense(1,activation='sigmoid'))
model_ann.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model_ann.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test),validation_batch_size=30)

In [ ]:
y_pre=model_ann.predict(x_sub,batch_size=30)
y_pre.shape

In [ ]:
 pre=[]
for val in y_pre:
    if val>=0.5:
        pre.append(1)
    else:
        pre.append(0)
            

In [ ]:
id1 =data_test['id']

In [ ]:
submission=pd.DataFrame({'id':id1,'tweet':pre})
submission.to_csv(r'/kaggle/working/df.csv',index=False)
